Run the following line if you don't have any of the libraries

In [ ]:
!pip install stata_kernel

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics, VAR
import data_prep_modified as dpm
import numpy as np

In [ ]:
data = pd.read_csv("./data/output.csv")

In [ ]:
#variable prep commands, replace snp500 with variable of choice
snp500delta = dpm.get_perc_return(data, column_name="Adj Close_SNP500")
snp500delta.set_index('Date', inplace=True)
#for some reason, I cant put the set_index command in the function because of some error

In [ ]:
snp500delta.head()

In [ ]:
ar_1 = AutoReg(snp500delta, lags =1).fit()
ar_1.summary()
ar_1.aic

Now using the Stata kernel to assess ADL models.

First import the data:

In [1]:
import delimited C:\Users\maple\Documents\GitHub\EC4304-forecasting\marcus\data\output.csv

(55 vars, 2,571 obs)


Next, create a business calendar and set the time index.

In [2]:
gen dcode = date(date, "DMY")
gen time = dofd(dcode)
format %td time
bcal create sp500, from(time) replace
drop time
gen time = bofd("sp500", dcode)
tsset time






  Business calendar sp500 (format %tbsp500):

    purpose:  

      range:  04jan2010  20mar2020
                 18266      21994   in %td units
                     0       2570   in %tbsp500 units

     center:  04jan2010
                 18266              in %td units
                     0              in %tbsp500 units

    omitted:     1,158              days
                   113.4            approx. days/year

   included:     2,571              days
                   251.8            approx. days/year

  Notes:

    business calendar file sp500.stbcal saved



        time variable:  time, 0 to 2570
                delta:  1 unit


Prepare the Adjusted Close variables for analysis by generating the percentage change variable.

In [3]:
foreach x of varlist adjclose_* {
    gen `x'_delta = 100*(log(`x') - log(L.`x'))
        } 


(1 missing value generated)
(1 missing value generated)
(1 missing value generated)
(1 missing value generated)
(1 missing value generated)
(1 missing value generated)
(1 missing value generated)
(1 missing value generated)
(1 missing value generated)


In [4]:
label variable adjclose_pharm_delta pharm
label variable adjclose_semicon_delta semicon
label variable adjclose_industrial_delta industrial
label variable adjclose_energy_delta energy
label variable adjclose_financial_delta financial
label variable adjclose_tech_delta tech
label variable adjclose_utilities_delta utilities
label variable adjclose_consumer_delta consumer
rename adjclose_snp500_delta adjclose_snp500_change

Check AIC of Benchmark Models

In [5]:
reg adjclose_snp500_change L.adjclose_snp500_change if time>5
estat ic



      Source |       SS           df       MS      Number of obs   =     2,565
-------------+----------------------------------   F(1, 2563)      =     59.30
       Model |  64.2247496         1  64.2247496   Prob > F        =    0.0000
    Residual |  2775.86157     2,563  1.08305172   R-squared       =    0.0226
-------------+----------------------------------   Adj R-squared   =    0.0222
       Total |  2840.08632     2,564  1.10767797   Root MSE        =    1.0407

-------------------------------------------------------------------------------
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.1509074   .0195967    -7.70   0.000    -.1893345   -.1124804
              |
        _cons |   .0315862   .0205564     1.54   0.125    -.0087226    .0718951
-------------------------------------------------------------------------------


Akaike's 

In [6]:
arima adjclose_snp500_change if time>5, ma(1/2)
estat ic



(setting optimization to BHHH)
Iteration 0:   log likelihood = -3733.0501  
Iteration 1:   log likelihood = -3733.0061  
Iteration 2:   log likelihood = -3732.9919  
Iteration 3:   log likelihood =  -3732.986  
Iteration 4:   log likelihood = -3732.9822  
(switching optimization to BFGS)
Iteration 5:   log likelihood = -3732.9795  
Iteration 6:   log likelihood = -3732.9742  
Iteration 7:   log likelihood = -3732.9737  
Iteration 8:   log likelihood = -3732.9726  
Iteration 9:   log likelihood = -3732.9726  
Iteration 10:  log likelihood = -3732.9726  

ARIMA regression

Sample:  6 - 2570                               Number of obs     =       2565
                                                Wald chi2(2)      =     637.97
Log likelihood = -3732.973                      Prob > chi2       =     0.0000

-------------------------------------------------------------------------------
              |                 OPG
adjclose_sn~e |      Coef.   Std. Err.      z    P>|z|     [95% Co

In [15]:
arima adjclose_snp500_change if time>5, arima(1,0,1)
estat ic



(setting optimization to BHHH)
Iteration 0:   log likelihood = -3735.2629  
Iteration 1:   log likelihood = -3734.8061  
Iteration 2:   log likelihood = -3734.6175  
Iteration 3:   log likelihood = -3734.5793  
Iteration 4:   log likelihood = -3734.5486  
(switching optimization to BFGS)
Iteration 5:   log likelihood = -3734.5265  
Iteration 6:   log likelihood = -3734.4945  
Iteration 7:   log likelihood = -3734.4855  
Iteration 8:   log likelihood = -3734.4849  
Iteration 9:   log likelihood = -3734.4841  
Iteration 10:  log likelihood =  -3734.484  
Iteration 11:  log likelihood =  -3734.484  

ARIMA regression

Sample:  6 - 2570                               Number of obs     =       2565
                                                Wald chi2(2)      =    1170.66
Log likelihood = -3734.484                      Prob > chi2       =     0.0000

-------------------------------------------------------------------------------
              |                 OPG
adjclose_sn~e |      

In [16]:
arima adjclose_snp500_change if time>5, arima(2,0,2)
estat ic



(setting optimization to BHHH)
Iteration 0:   log likelihood = -3733.3349  
Iteration 1:   log likelihood = -3732.7963  
Iteration 2:   log likelihood = -3732.7397  
Iteration 3:   log likelihood = -3732.7112  
Iteration 4:   log likelihood = -3732.6933  
(switching optimization to BFGS)
Iteration 5:   log likelihood = -3732.6822  
Iteration 6:   log likelihood = -3732.6643  
Iteration 7:   log likelihood =  -3732.662  
Iteration 8:   log likelihood = -3732.6593  
Iteration 9:   log likelihood =  -3732.659  
Iteration 10:  log likelihood = -3732.6589  
Iteration 11:  log likelihood = -3732.6589  
Iteration 12:  log likelihood = -3732.6589  
Iteration 13:  log likelihood = -3732.6589  

ARIMA regression

Sample:  6 - 2570                               Number of obs     =       2565
                                                Wald chi2(4)      =     838.55
Log likelihood = -3732.659                      Prob > chi2       =     0.0000

-----------------------------------------------

Check the Granger Causality of the exogenous variables.

In [7]:
foreach x of varlist *_delta {
    newey adjclose_snp500_change L(1/2).adjclose_snp500_change L(1/5).`x', lag(5)
    testparm L(1/5).`x'
    }



Regression with Newey-West standard errors      Number of obs     =      2,565
maximum lag: 5                                  F(  7,      2557) =       2.16
                                                Prob > F          =     0.0352

-------------------------------------------------------------------------------
              |             Newey-West
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.1472354   .0721952    -2.04   0.042    -.2888025   -.0056684
          L2. |   .1223788   .0535325     2.29   0.022     .0174074    .2273503
              |
adjclose_ph~a |
          L1. |   .0104501   .0383502     0.27   0.785    -.0647506    .0856507
          L2. |  -.0553513   .0356548    -1.55   0.121    -.1252664    .0145639
          L3. |  -.0041683   .0456668    -0.09   0.927     -.093716    .0853794
          L4. |   .0033646   .031

--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.0724758   .0717541    -1.01   0.313    -.2131779    .0682263
          L2. |   .2103047   .1084011     1.94   0.052    -.0022582    .4228677
              |
adjclose_te~a |
          L1. |  -.0643614   .0728364    -0.88   0.377    -.2071857    .0784629
          L2. |  -.1208798   .0678389    -1.78   0.075    -.2539046     .012145
          L3. |   .0023642   .0438587     0.05   0.957    -.0836381    .0883664
          L4. |  -.0186519   .0300655    -0.62   0.535    -.0776072    .0403033
          L5. |   -.034943   .0363629    -0.96   0.337    -.1062468    .0363608
              |
        _cons |   .0357176   .0215391     1.66   0.097    -.0065182    .0779535
-------------------------------------------------------------------------------

 ( 1)  L.adjclose_tech_delta = 0
 ( 2)  L2.adjclose_tech_delta = 0
 ( 3)  L3.adjclose_tech_delta = 0
 ( 4)  L4.adjclose_tech_delta = 0


Check corrgram of the variables

In [18]:
foreach x of varlist *_delta {
    corrgram `x'
    }



                                          -1       0       1 -1       0       1
 LAG       AC       PAC      Q     Prob>Q  [Autocorrelation]  [Partial Autocor]
-------------------------------------------------------------------------------
1       -0.0686  -0.0688   12.112  0.0005          |                  |        
2        0.0555   0.0511   20.037  0.0000          |                  |        
3       -0.0122  -0.0055   20.422  0.0001          |                  |        
4       -0.0016  -0.0052   20.429  0.0004          |                  |        
5        0.0009   0.0007   20.431  0.0010          |                  |        
6       -0.0037  -0.0029   20.467  0.0023          |                  |        
7        0.0329   0.0357   23.255  0.0015          |                  |        
8       -0.0704  -0.0767   36.056  0.0000          |                  |        
9        0.0304   0.0278   38.445  0.0000          |                  |        
10       0.0019   0.0084   38.454  0.0

15      -0.0186  -0.0194   84.883  0.0000          |                  |        
16       0.0314   0.0466   87.431  0.0000          |                  |        
17       0.0335   0.0449   90.337  0.0000          |                  |        
18      -0.0245  -0.0341   91.895  0.0000          |                  |        
19      -0.0282  -0.0495    93.96  0.0000          |                  |        
20       0.0039   0.0004   93.999  0.0000          |                  |        
21      -0.0027   0.0084   94.018  0.0000          |                  |        
22      -0.0367  -0.0444   97.517  0.0000          |                  |        
23       0.0125   0.0076   97.922  0.0000          |                  |        
24       0.0148   0.0201   98.488  0.0000          |                  |        
25      -0.0510  -0.0562   105.25  0.0000          |                  |        
26      -0.0263  -0.0313   107.04  0.0000          |                  |        
27       0.0196   0.0231   108.04  0.000

32      -0.0131  -0.0210   153.23  0.0000          |                  |        
33       0.0051   0.0041    153.3  0.0000          |                  |        
34      -0.0085  -0.0023   153.49  0.0000          |                  |        
35      -0.0036   0.0059   153.52  0.0000          |                  |        
36       0.0080   0.0025   153.69  0.0000          |                  |        
37       0.0269   0.0266   155.58  0.0000          |                  |        
38       0.0037   0.0215   155.62  0.0000          |                  |        
39       0.0259   0.0305   157.37  0.0000          |                  |        
40       0.0446   0.0465   162.57  0.0000          |                  |        

                                          -1       0       1 -1       0       1
 LAG       AC       PAC      Q     Prob>Q  [Autocorrelation]  [Partial Autocor]
-------------------------------------------------------------------------------
1       -0.1456  -0.1465   54.535  0.00

6       -0.0095  -0.0112   44.682  0.0000          |                  |        
7        0.0610   0.0674   54.267  0.0000          |                  |        
8       -0.0720  -0.0749   67.626  0.0000          |                  |        
9        0.0435   0.0355   72.516  0.0000          |                  |        
10      -0.0047   0.0084   72.573  0.0000          |                  |        
11       0.0283   0.0234   74.646  0.0000          |                  |        
12       0.0278   0.0431   76.644  0.0000          |                  |        
13       0.0033  -0.0002   76.672  0.0000          |                  |        
14      -0.0293  -0.0350   78.888  0.0000          |                  |        
15      -0.0188  -0.0183   79.806  0.0000          |                  |        
16       0.0197   0.0228   80.809  0.0000          |                  |        
17       0.0301   0.0419   83.152  0.0000          |                  |        
18      -0.0034  -0.0047   83.183  0.000

Check using robust standard errors instead of HAC standard errors.

In [19]:
foreach x of varlist *_delta {
    reg adjclose_snp500_change L(1/2).adjclose_snp500_change L(1/5).`x', r
    testparm L(1/5).`x'
    }



Linear regression                               Number of obs     =      2,565
                                                F(7, 2557)        =       1.86
                                                Prob > F          =     0.0720
                                                R-squared         =     0.0303
                                                Root MSE          =     1.0378

-------------------------------------------------------------------------------
              |               Robust
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.1472354   .0618457    -2.38   0.017    -.2685081   -.0259627
          L2. |   .1223788   .0620651     1.97   0.049     .0006758    .2440819
              |
adjclose_ph~a |
          L1. |   .0104501   .0373257     0.28   0.780    -.0627417    .0836418
          L2. |  -.0553513   .0375907

-------------------------------------------------------------------------------

 ( 1)  L.adjclose_financial_delta = 0
 ( 2)  L2.adjclose_financial_delta = 0
 ( 3)  L3.adjclose_financial_delta = 0
 ( 4)  L4.adjclose_financial_delta = 0
 ( 5)  L5.adjclose_financial_delta = 0

       F(  5,  2557) =    1.47
            Prob > F =    0.1949

Linear regression                               Number of obs     =      2,565
                                                F(7, 2557)        =       2.19
                                                Prob > F          =     0.0321
                                                R-squared         =     0.0335
                                                Root MSE          =     1.0361

-------------------------------------------------------------------------------
              |               Robust
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+---------------------------------------------------

Summary of results:

|            | HAC Standard Error | Robust Standard Error |
|------------|:------------------:|:---------------------:|
| pharm      |       0.7631       |         0.8043        |
| semicon    |       0.3060       |         0.2033        |
| industrial |       0.0729       |         0.0269        |
| energy     |       0.0356       |         0.1552        |
| financial  |       0.1494       |         0.1949        |
| tech       |       0.2243       |         0.1720        |
| utilities  |       0.0845       |         0.2069        |
| consumer   |       0.2145       |         0.2399        |

Based on the granger causality test, it appears that only industrial seems to contribute to snp500.

Perform ADL regressions and store estimates.

In [20]:
reg adjclose_snp500_change L(1/2).adjclose_snp500_change if time>5
estimates store ar2
foreach x of varlist *_delta {
    local varlabel : variable label `x'
    reg adjclose_snp500_change L(1/2).adjclose_snp500_change L.`x' if time>5
    estimates store adl`varlabel'21
    reg adjclose_snp500_change L(1/2).adjclose_snp500_change L(1/2).`x' if time>5
    estimates store adl`varlabel'22
    reg adjclose_snp500_change L(1/2).adjclose_snp500_change L(1/3).`x' if time>5
    estimates store adl`varlabel'23
    reg adjclose_snp500_change L(1/2).adjclose_snp500_change L(1/4).`x' if time>5
    estimates store adl`varlabel'24
    reg adjclose_snp500_change L(1/2).adjclose_snp500_change L(1/5).`x' if time>5
    estimates store adl`varlabel'25
        }



      Source |       SS           df       MS      Number of obs   =     2,565
-------------+----------------------------------   F(2, 2562)      =     37.89
       Model |  81.5837786         2  40.7918893   Prob > F        =    0.0000
    Residual |  2758.50254     2,562  1.07669888   R-squared       =    0.0287
-------------+----------------------------------   Adj R-squared   =    0.0280
       Total |  2840.08632     2,564  1.10767797   Root MSE        =    1.0376

-------------------------------------------------------------------------------
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.1389882   .0197634    -7.03   0.000    -.1777421   -.1002344
          L2. |   .0793573   .0197638     4.02   0.000     .0406026     .118112
              |
        _cons |   .0289443   .0205066     1.41   0.158    -.0112668    .0691555
-----------


-------------------------------------------------------------------------------
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.2192482   .0344151    -6.37   0.000    -.2867325    -.151764
          L2. |   .0805003   .0197406     4.08   0.000     .0417912    .1192094
              |
adjclose_se~a |
          L1. |   .0665144   .0233652     2.85   0.004     .0206978    .1123309
              |
        _cons |   .0273493   .0204859     1.34   0.182    -.0128212    .0675199
-------------------------------------------------------------------------------

      Source |       SS           df       MS      Number of obs   =     2,565
-------------+----------------------------------   F(4, 2560)      =     21.02
       Model |  90.3067962         4   22.576699   Prob > F        =    0.0000
    Residual |  2749.77952     2,560  1.07413263   R-squa

          L2. |   .0000987   .0598726     0.00   0.999    -.1173049    .1175023
              |
adjclose_in~a |
          L1. |   .2037725    .051838     3.93   0.000     .1021238    .3054213
          L2. |   .0629352   .0515756     1.22   0.222    -.0381989    .1640694
              |
        _cons |   .0295189   .0204472     1.44   0.149    -.0105759    .0696137
-------------------------------------------------------------------------------

      Source |       SS           df       MS      Number of obs   =     2,565
-------------+----------------------------------   F(5, 2559)      =     18.73
       Model |  100.274708         5  20.0549417   Prob > F        =    0.0000
    Residual |  2739.81161     2,559  1.07065714   R-squared       =    0.0353
-------------+----------------------------------   Adj R-squared   =    0.0334
       Total |  2840.08632     2,564  1.10767797   Root MSE        =    1.0347

----------------------------------------------------------------------------

          L3. |   .0289791   .0136362     2.13   0.034       .00224    .0557182
              |
        _cons |   .0361149   .0205267     1.76   0.079    -.0041358    .0763655
-------------------------------------------------------------------------------

      Source |       SS           df       MS      Number of obs   =     2,565
-------------+----------------------------------   F(6, 2558)      =     16.19
       Model |  103.885053         6  17.3141755   Prob > F        =    0.0000
    Residual |  2736.20127     2,558  1.06966429   R-squared       =    0.0366
-------------+----------------------------------   Adj R-squared   =    0.0343
       Total |  2840.08632     2,564  1.10767797   Root MSE        =    1.0342

-------------------------------------------------------------------------------
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1

              |
        _cons |   .0287179   .0205036     1.40   0.161    -.0114874    .0689233
-------------------------------------------------------------------------------

      Source |       SS           df       MS      Number of obs   =     2,565
-------------+----------------------------------   F(7, 2557)      =     12.86
       Model |  96.5551706         7  13.7935958   Prob > F        =    0.0000
    Residual |  2743.53115     2,557  1.07294922   R-squared       =    0.0340
-------------+----------------------------------   Adj R-squared   =    0.0314
       Total |  2840.08632     2,564  1.10767797   Root MSE        =    1.0358

-------------------------------------------------------------------------------
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.0793248   .0375822    -2.11   0.035    -.1530193   -.0056302
          L2

              |
        _cons |   .0357176   .0206105     1.73   0.083    -.0046973    .0761325
-------------------------------------------------------------------------------

      Source |       SS           df       MS      Number of obs   =     2,565
-------------+----------------------------------   F(3, 2561)      =     28.38
       Model |  91.3698142         3  30.4566047   Prob > F        =    0.0000
    Residual |   2748.7165     2,561  1.07329813   R-squared       =    0.0322
-------------+----------------------------------   Adj R-squared   =    0.0310
       Total |  2840.08632     2,564  1.10767797   Root MSE        =     1.036

-------------------------------------------------------------------------------
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.0938973   .0247457    -3.79   0.000     -.142421   -.0453736
          L2

       Model |  94.5730351         4  23.6432588   Prob > F        =    0.0000
    Residual |  2745.51328     2,560  1.07246613   R-squared       =    0.0333
-------------+----------------------------------   Adj R-squared   =    0.0318
       Total |  2840.08632     2,564  1.10767797   Root MSE        =    1.0356

-------------------------------------------------------------------------------
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.3128614   .0548446    -5.70   0.000    -.4204057    -.205317
          L2. |   .0317112   .0555304     0.57   0.568    -.0771779    .1406002
              |
adjclose_co~a |
          L1. |   .1716743   .0509154     3.37   0.001     .0718348    .2715139
          L2. |   .0444706   .0515755     0.86   0.389    -.0566634    .1456045
              |
        _cons |   .0250143   .0205063     1.22   0.223    -

Compare AIC values

In [21]:
estimates stats ar2 adlpharm21 adlpharm22 adlpharm23 adlpharm24 adlpharm25 adlsemicon21 adlsemicon22 adlsemicon23 adlsemicon24 adlsemicon25 adlindustrial21 adlindustrial22 adlindustrial23 adlindustrial24 adlindustrial25 adlenergy21 adlenergy22 adlenergy23 adlenergy24 adlenergy25 adlfinancial21 adlfinancial22 adlfinancial23 adlfinancial24 adlfinancial25 adltech21 adltech22 adltech23 adltech24 adltech25 adlutilities21 adlutilities22 adlutilities23 adlutilities24 adlutilities25 adlconsumer21 adlconsumer22 adlconsumer23 adlconsumer24 adlconsumer25


Akaike's information criterion and Bayesian information criterion

-----------------------------------------------------------------------------
       Model |        Obs  ll(null)  ll(model)      df         AIC        BIC
-------------+---------------------------------------------------------------
         ar2 |      2,565 -3770.233  -3732.853       3    7471.706   7489.255
  adlpharm21 |      2,565 -3770.233  -3732.828       4    7473.657   7497.056
  adlpharm22 |      2,565 -3770.233  -3731.536       5    7473.071    7502.32
  adlpharm23 |      2,565 -3770.233  -3731.504       6    7475.008   7510.106
  adlpharm24 |      2,565 -3770.233  -3731.482       7    7476.964   7517.912
  adlpharm25 |      2,565 -3770.233  -3730.837       8    7477.673   7524.471
adlsemicon21 |      2,565 -3770.233  -3728.801       4    7465.602   7489.001
adlsemicon22 |      2,565 -3770.233  -3728.791       5    7467.582    7496.83
adlsemicon23 |      2,565 -3770.233  -3728.082       6    7468.165   7503.

Based on the AIC, the lag orders should be 2 1 1 3 5 2 3 1. We drop pharm as an exogenous variable.

Note that the AIC for the ADL(2,1) model only containing the industrial variable, 7458.196, is the lowest AIC value.

Next we should test the combinations of different exogenous variables to find the best model. Due to the number of exogenous variables, manually trying out all the combinations would be a labourious, time consuming task. Hence we will use the ardl package to help us.

Due to the pecularities of the package, generate the first lag of each exogenous variable.

In [5]:
foreach x of varlist *_delta {
    gen `x'_lag = L.`x'
    }


(2 missing values generated)
(2 missing values generated)
(2 missing values generated)
(2 missing values generated)
(2 missing values generated)
(2 missing values generated)
(2 missing values generated)
(2 missing values generated)


In [12]:
ardl adjclose_snp500_change adjclose_semicon_delta_lag adjclose_industrial_delta_lag adjclose_energy_delta_lag adjclose_financial_delta_lag adjclose_tech_delta_lag adjclose_utilities_delta_lag adjclose_consumer_delta_lag, maxcombs(8398080) lag(2 . . . . . . .) maxlag(4) aic


ARDL(2,0,0,2,4,0,2,1) regression

Sample:         6 -      2570                   Number of obs     =      2,565
                                                F(  18,   2546)   =      10.02
                                                Prob > F          =     0.0000
                                                R-squared         =     0.0661
                                                Adj R-squared     =     0.0595
Log likelihood = -3682.4731                     Root MSE          =     1.0207

-------------------------------------------------------------------------------
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.2265372   .1459558    -1.55   0.121    -.5127415     .059667
          L2. |  -.2545655   .0828404    -3.07   0.002     -.417007    -.092124
              |
adjclose_se~g |   .0778841   .0274258     2.84   0.005    

In [13]:
estat ic


Akaike's information criterion and Bayesian information criterion

-----------------------------------------------------------------------------
       Model |        Obs  ll(null)  ll(model)      df         AIC        BIC
-------------+---------------------------------------------------------------
           . |      2,565 -3770.233  -3682.473      19    7402.946   7514.091
-----------------------------------------------------------------------------
               Note: N=Obs used in calculating BIC; see [R] BIC note.


The combined model is apparently a better fit than the ADL(2,1) model only using industrial as an exogenous variable.

However, our goal is to see which model is the best for forecasting. Rather than just looking at AIC values, we should also look at how well the model fits for in-sample, pseudo-out of sample and out of sample predictions.

The models which we will look at are as follows:

AR(2)

ADL(2,1) with industrial as the exogenous variable

combined(2,1,1,3,5,1,3,2)

We will look at the time period of the next 5 business days as an out of sample fit.

In [6]:
tsappend, add(5)

In [7]:
reg adjclose_snp500_change L(1/2).adjclose_snp500_change if time<2571
predict ar2in



      Source |       SS           df       MS      Number of obs   =     2,568
-------------+----------------------------------   F(2, 2565)      =     37.92
       Model |  81.5673066         2  40.7836533   Prob > F        =    0.0000
    Residual |  2758.74683     2,565  1.07553483   R-squared       =    0.0287
-------------+----------------------------------   Adj R-squared   =    0.0280
       Total |  2840.31414     2,567   1.1064722   Root MSE        =    1.0371

-------------------------------------------------------------------------------
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |    -.13892   .0197518    -7.03   0.000    -.1776511   -.1001888
          L2. |   .0794258   .0197522     4.02   0.000     .0406939    .1181578
              |
        _cons |   .0292586   .0204839     1.43   0.153     -.010908    .0694251
-----------

In [8]:
forvalues p=1/5{
    local q = `p'+1
    qui reg adjclose_snp500_change L(`p'/`q').adjclose_snp500_change
    predict ar2`p'
    predict ar2sf`p', stdf
    gen ar2`p'L68 = ar2`p'-0.995*ar2sf`p'
    gen ar2`p'U68 = ar2`p'+0.995*ar2sf`p'
    gen ar2`p'L50 = ar2`p'-0.675*ar2sf`p'
    gen ar2`p'U50 = ar2`p'+0.675*ar2sf`p'
}


(option xb assumed; fitted values)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(option xb assumed; fitted values)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(option xb assumed; fitted values)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(option xb assumed; fitted values)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(option xb assumed; fitted values)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing v

In [9]:
egen ar2p2=rowfirst(ar21 ar22 ar23 ar24 ar25) if time>=2571
egen ar2pL68=rowfirst(ar21L68 ar22L68 ar23L68 ar24L68 ar25L68) if time>=2571
egen ar2pU68=rowfirst(ar21U68 ar22U68 ar23U68 ar24U68 ar25U68) if time>=2571
egen ar2pL50=rowfirst(ar21L50 ar22L50 ar23L50 ar24L50 ar25L50) if time>=2571
egen ar2pU50=rowfirst(ar21U50 ar22U50 ar23U50 ar24U50 ar25U50) if time>=2571


(2571 missing values generated)

(2571 missing values generated)

(2571 missing values generated)

(2571 missing values generated)

(2571 missing values generated)


In [10]:
reg adjclose_snp500_change L(1/2).adjclose_snp500_change L.adjclose_industrial_delta if time<2571
predict adl21in



      Source |       SS           df       MS      Number of obs   =     2,568
-------------+----------------------------------   F(3, 2564)      =     30.64
       Model |  98.3151126         3  32.7717042   Prob > F        =    0.0000
    Residual |  2741.99902     2,564   1.0694224   R-squared       =    0.0346
-------------+----------------------------------   Adj R-squared   =    0.0335
       Total |  2840.31414     2,567   1.1064722   Root MSE        =    1.0341

-------------------------------------------------------------------------------
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.3636112   .0600973    -6.05   0.000    -.4814553    -.245767
          L2. |   .0690277   .0198705     3.47   0.001     .0300638    .1079916
              |
adjclose_in~a |
          L1. |   .2046256   .0517078     3.96   0.000     .1032324    .306

In [11]:
forvalues p=1/5{
    local q = `p'+1
    qui reg adjclose_snp500_change L(`p'/`q').adjclose_snp500_change L`p'.adjclose_industrial_delta
    predict adl21`p'
    predict adl21sf`p', stdf
    gen adl21`p'L68 = adl21`p'-0.995*adl21sf`p'
    gen adl21`p'U68 = adl21`p'+0.995*adl21sf`p'
    gen adl21`p'L50 = adl21`p'-0.675*adl21sf`p'
    gen adl21`p'U50 = adl21`p'+0.675*adl21sf`p'
}


(option xb assumed; fitted values)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(option xb assumed; fitted values)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(option xb assumed; fitted values)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(option xb assumed; fitted values)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(option xb assumed; fitted values)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing values generated)
(7 missing v

In [12]:
egen adl21p2=rowfirst(adl211 adl212 adl213 adl214 adl215) if time>=2571
egen adl21pL68=rowfirst(adl211L68 adl212L68 adl213L68 adl214L68 adl215L68) if time>=2571
egen adl21pU68=rowfirst(adl211U68 adl212U68 adl213U68 adl214U68 adl215U68) if time>=2571
egen adl21pL50=rowfirst(adl211L50 adl212L50 adl213L50 adl214L50 adl215L50) if time>=2571
egen adl21pU50=rowfirst(adl211U50 adl212U50 adl213U50 adl214U50 adl215U50) if time>=2571


(2571 missing values generated)

(2571 missing values generated)

(2571 missing values generated)

(2571 missing values generated)

(2571 missing values generated)


In [14]:
reg adjclose_snp500_change L(1/2).adjclose_snp500_change L.adjclose_semicon_delta L.adjclose_industrial_delta L(1/3).adjclose_energy_delta L(1/5).adjclose_financial_delta L.adjclose_tech_delta L(1/3).adjclose_utilities_delta L(1/2).adjclose_consumer_delta if time<2571
predict combin



      Source |       SS           df       MS      Number of obs   =     2,565
-------------+----------------------------------   F(18, 2546)     =     10.02
       Model |  187.843986        18   10.435777   Prob > F        =    0.0000
    Residual |  2652.24233     2,546  1.04172912   R-squared       =    0.0661
-------------+----------------------------------   Adj R-squared   =    0.0595
       Total |  2840.08632     2,564  1.10767797   Root MSE        =    1.0207

-------------------------------------------------------------------------------
adjclose_sn~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
adjclose_sn~e |
          L1. |  -.2265372   .1459558    -1.55   0.121    -.5127415     .059667
          L2. |  -.2545655   .0828404    -3.07   0.002     -.417007    -.092124
              |
adjclose_se~a |
          L1. |   .0778841   .0274258     2.84   0.005     .0241049    .131

In [15]:
forvalues p=1/5{
    local q = `p'+1
    local s = `p'+2
    local u = `p'+4
    qui reg adjclose_snp500_change L(`p'/`q').adjclose_snp500_change L`p'.adjclose_semicon_delta L`p'.adjclose_industrial_delta L(`p'/`s').adjclose_energy_delta L(`p'/`u').adjclose_financial_delta L`p'.adjclose_tech_delta L(`p'/`s').adjclose_utilities_delta L(`p'/`q').adjclose_consumer_delta
    predict comb`p'
    predict combsf`p', stdf
    gen comb`p'L68 = comb`p'-0.995*combsf`p'
    gen comb`p'U68 = comb`p'+0.995*combsf`p'
    gen comb`p'L50 = comb`p'-0.675*combsf`p'
    gen comb`p'U50 = comb`p'+0.675*combsf`p'
}


(option xb assumed; fitted values)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(option xb assumed; fitted values)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(option xb assumed; fitted values)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(option xb assumed; fitted values)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(option xb assumed; fitted values)
(10 missing values generated)
(10 missing values generated)
(10 missing values generated)
(10 missing va

In [16]:
egen combp2=rowfirst(comb1 comb2 comb3 comb4 comb5) if time>=2571
egen combpL68=rowfirst(comb1L68 comb2L68 comb3L68 comb4L68 comb5L68) if time>=2571
egen combpU68=rowfirst(comb1U68 comb2U68 comb3U68 comb4U68 comb5U68) if time>=2571
egen combpL50=rowfirst(comb1L50 comb2L50 comb3L50 comb4L50 comb5L50) if time>=2571
egen combpU50=rowfirst(comb1U50 comb2U50 comb3U50 comb4U50 comb5U50) if time>=2571


(2571 missing values generated)

(2571 missing values generated)

(2571 missing values generated)

(2571 missing values generated)

(2571 missing values generated)


In [ ]:
tsline adjclose_snp500_change ar2in ar2p2 ar2pL50 ar2pU50 ar2pL68 ar2pU68 if time>=2545, title(adj_close_SnP_%chg 50% 68% fan with AR2 Model) lpattern (solid solid solid longdash longdash shortdash shortdash)

tsline adjclose_snp500_change adl21in adl21p2 adl21pL50 adl21pU50 adl21pL68 adl21pU68 if time>=2545, title(adj_close_SnP_%chg 50% 68% fan with ADL21 Model) lpattern (solid solid solid longdash longdash shortdash shortdash)

tsline adjclose_snp500_change combin combp2 combpL50 combpU50 combpL68 combpU68 if time>=2545, title(adj_close_SnP_%chg 50% 68% fan with COMB Model) lpattern (solid solid solid longdash longdash shortdash shortdash)

<img src="files/snp500changear2combfchart.png" style="width: 500px;"/>

<img src="files/snp500changeadl21combfchart.png" style="width: 500px;"/>

<img src="files/snp500changecombcombfchart.png" style="width: 500px;"/>

All 3 models appear to do a better job at predicting the in sample fit for later time periods. 